In [125]:
import pandas as pd
import numpy as np
import seaborn as sns

# ML Models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


import itertools

### Loading data

In [126]:
# TODO: Test on merged data from 1st and 2nd iteration



users_data = pd.read_json("proper_data/users2.json")
deliveries_data = pd.read_json("proper_data/deliveries2.json")
events_data = pd.read_json("proper_data/sessions2.json")
products_data = pd.read_json("proper_data/products2.json")

In [127]:
users_data

,user_id,name,city,street
0,102,Aurelia Malon,Police,pl. Brzoskwiniowa 11/53
1,103,Mateusz Kobel,Police,al. Wrocławska 10
2,104,Radosław Ratka,Mielec,pl. Nowa 89/04
3,105,Anastazja Oszust,Szczecin,ul. Częstochowska 80
4,106,Sylwia Nurek,Szczecin,al. Wiosenna 72
...,...,...,...,...
195,297,Andrzej Kreczmer,Szczecin,ul. Zdrojowa 43/11
196,298,Natan Świadek,Mielec,pl. Staffa 801
197,299,Jeremi Kajstura,Konin,pl. Wiklinowa 022
198,300,Gustaw Cofała,Gdynia,ulica Krakowska 847


In [128]:
deliveries_data

,purchase_id,purchase_timestamp,delivery_timestamp,delivery_company
0,20001,2021-03-26T06:19:44,2021-03-29T06:23:48,620
1,20002,2020-12-03T08:25:48,2020-12-07T08:27:30,360
2,20003,2021-03-04T02:46:03,2021-03-07T02:49:30,360
3,20004,2021-03-03T19:12:38,2021-03-06T19:13:32,620
4,20005,2020-11-04T23:03:37,2020-11-08T23:07:25,516
...,...,...,...,...
7176,27177,2021-03-27T12:44:44,2021-03-31T12:48:16,360
7177,27178,2021-02-25T01:12:12,2021-03-01T01:13:21,360
7178,27179,2021-02-28T09:50:14,2021-03-03T09:52:52,516
7179,27180,2021-03-04T04:53:21,2021-03-09T04:56:57,516


In [129]:
events_data

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2021-03-26 06:16:10,102,1281,VIEW_PRODUCT,0,NaN
1,100001,2021-03-26 06:18:47,102,1278,VIEW_PRODUCT,0,NaN
2,100001,2021-03-26 06:19:44,102,1278,BUY_PRODUCT,0,20001.0
3,100002,2020-12-03 08:25:15,102,1278,VIEW_PRODUCT,0,NaN
4,100002,2020-12-03 08:25:48,102,1278,BUY_PRODUCT,0,20002.0
...,...,...,...,...,...,...,...
40449,110787,2021-04-24 07:43:01,301,1004,VIEW_PRODUCT,0,NaN
40450,110787,2021-04-24 07:44:45,301,1013,VIEW_PRODUCT,0,NaN
40451,110787,2021-04-24 07:47:39,301,1006,VIEW_PRODUCT,0,NaN
40452,110787,2021-04-24 07:49:28,301,1007,VIEW_PRODUCT,0,NaN


In [130]:
products_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     319 non-null    int64  
 1   product_name   319 non-null    object 
 2   category_path  319 non-null    object 
 3   price          319 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.1+ KB


## Preprocessing

## Extracting features

### "Indexing features"

In [131]:
events_data['year'] = events_data.timestamp.dt.year
events_data['month'] = events_data.timestamp.dt.month


years = list(events_data.year.unique())
months = list(events_data.month.unique())
user_ids = list(events_data.user_id.unique())

triplets = []
for triplet in itertools.product(years, months, user_ids):
    triplets.append(triplet)
    
processed_data = pd.DataFrame(triplets, columns=['year', 'month', 'user_id'])
processed_data

,year,month,user_id
0,2021,3,102
1,2021,3,103
2,2021,3,104
3,2021,3,105
4,2021,3,106
...,...,...,...
4795,2020,10,297
4796,2020,10,298
4797,2020,10,299
4798,2020,10,300


### Number of all events per user per month

In [132]:
all_events = events_data.groupby(['user_id', 'year', 'month']).aggregate({"session_id": "count"}) \
    .rename(columns={"session_id": "all_sessions"}) \
    .reset_index()

### Number of buying events per user per month

In [133]:
buying_events = events_data[events_data['event_type'] == 'BUY_PRODUCT'] \
    .groupby(['user_id', 'year', 'month']) \
    .aggregate({"session_id": "count"}) \
    .rename(columns={"session_id": "buying_sessions"}) \
    .reset_index()

### Buying ratio

In [134]:
events_ratio = pd.merge(all_events, buying_events, how="left", on=["year", "month", "user_id"])

# all_events
# buying_events

events_ratio['buying_sessions'].fillna(0, inplace=True)

events_ratio['buying_ratio'] = round(events_ratio['buying_sessions'] / events_ratio['all_sessions'], 4)


# merging with processed_data
processed_data = pd.merge(processed_data, events_ratio.drop("all_sessions", axis=1), how='left', on=['year', 'month', 'user_id'])
processed_data['buying_sessions'].fillna(0, inplace=True)
processed_data['buying_ratio'].fillna(0, inplace=True)

In [135]:
processed_data

,year,month,user_id,buying_sessions,buying_ratio
0,2021,3,102,5.0,0.2381
1,2021,3,103,25.0,0.2358
2,2021,3,104,5.0,0.1316
3,2021,3,105,6.0,0.3000
4,2021,3,106,14.0,0.1842
...,...,...,...,...,...
4795,2020,10,297,8.0,0.1778
4796,2020,10,298,2.0,0.2500
4797,2020,10,299,0.0,0.0000
4798,2020,10,300,0.0,0.0000


### Data Cleaner

In [136]:
def clean_customers(customers_data):
    customers_data.info()
    pass

In [137]:
def clean_products(products_data):
    products_data.info()
    pass

In [138]:
def clean_deliveries(deliveries_data):
    info_df = deliveries_data.info()
    
    if any([info_df[feature]['Non-Null Count'] != len(deliveries_data) for feature in info_df]):
        print()
    pass

In [139]:
def clean_events():
    pass

### Constructing target variable - how much money the user spent per month

In [140]:
buying_sessions = events_data[events_data['event_type'] == "BUY_PRODUCT"]

In [141]:
deals = pd.merge(buying_sessions, products_data, how="left", on=['product_id'])

In [142]:
deals['final_price'] = deals['price'] * (1 - deals['offered_discount'] * 0.01)

In [143]:
monthly_deals = deals.groupby(['year', 'month', 'user_id']) \
    .aggregate({"final_price": "sum"}) \
    .rename(columns={"final_price": "money_monthly"}) \
    .reset_index()

In [144]:
processed_data = pd.merge(processed_data, monthly_deals, how='left', on=['year', 'month', 'user_id'])

In [145]:
processed_data

,year,month,user_id,buying_sessions,buying_ratio,money_monthly
0,2021,3,102,5.0,0.2381,5863.1530
1,2021,3,103,25.0,0.2358,17784.3495
2,2021,3,104,5.0,0.1316,11720.3560
3,2021,3,105,6.0,0.3000,7220.2930
4,2021,3,106,14.0,0.1842,13131.9625
...,...,...,...,...,...,...
4795,2020,10,297,8.0,0.1778,3586.5420
4796,2020,10,298,2.0,0.2500,2600.1690
4797,2020,10,299,0.0,0.0000,NaN
4798,2020,10,300,0.0,0.0000,NaN


In [147]:
processed_data['money_monthly'].fillna(0, inplace=True)

In [148]:
processed_data

,year,month,user_id,buying_sessions,buying_ratio,money_monthly
0,2021,3,102,5.0,0.2381,5863.1530
1,2021,3,103,25.0,0.2358,17784.3495
2,2021,3,104,5.0,0.1316,11720.3560
3,2021,3,105,6.0,0.3000,7220.2930
4,2021,3,106,14.0,0.1842,13131.9625
...,...,...,...,...,...,...
4795,2020,10,297,8.0,0.1778,3586.5420
4796,2020,10,298,2.0,0.2500,2600.1690
4797,2020,10,299,0.0,0.0000,0.0000
4798,2020,10,300,0.0,0.0000,0.0000
